### Pomožne funkcije pri izdelavi pajka
 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

WEB_PAGE_ADDRESS = "https://e-prostor.gov.si"
html_links = []

chrome_options = Options()
# If you comment the following line, a browser will show ...
#chrome_options.add_argument("--headless")

#Adding a specific user agent
chrome_options.add_argument("user-agent=fri-ieps-TEST")

print(f"Retrieving web page URL '{WEB_PAGE_ADDRESS}'")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options)
driver.get(WEB_PAGE_ADDRESS)


links = driver.find_elements(By.TAG_NAME, "a")

for link in links:
    raw_links = link.get_attribute("href")
    if raw_links is not None:
        if raw_links[0:4] == "http":
            #sprinta like strani
            html_links.append("link")
        else:
            #sprinta ostalo
            print(raw_links)


driver.close()

Retrieving web page URL 'https://e-prostor.gov.si'


[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:00<00:00, 11.4MB/s]


mailto:http://www.nlb.si/?doc=5458
mailto:http://postarca.posta.si/
mailto:http://www.nlb.si/?doc=5458
mailto:http://postarca.posta.si/
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
mailto:http://www.nlb.si/?doc=5458
mailto:http://postarca.posta.si/
javascript:linkTo_UnCryptMailto(%27kygjrm8gldmYqgrpylcr%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8gldmYqgrpylcr%2Cq%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8gldmYqgrpylcr%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8gldmYqgrpylcr%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8gldmYqgrpylcr%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27);
javascript:linkTo_UnCryptMailto(%27kygjrm8nmkma%2CesYemt%2Cqg%27)

In [5]:
from urllib.parse import urlparse

websites = set()
web = "https://docs.python.org/3/library/"

# Odstrani duplikate glede na url:
#    HTTP vs HTTPS
#    www vs non-www
#    Trailing slash: example.com vs example.com/
def is_unique(website):
    # Strip of the last bit
    parsed = urlparse(website)
    path = parsed.path[:-1] if (parsed.path[-1] == "/") else parsed.path
    url = parsed.hostname + path
    if url in websites:
        return False
    websites.add(url)
    print(url)
    return True

is_unique(web)

docs.python.org/3/library


True

In [4]:
import urllib 
from bs4 import BeautifulSoup
from polyglot.text import Text
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [3]:
def recognizeLang(text):
    """
    Pomožna funkcija, ki pretvori pravilen zapis jezika
    """
    textObj = Text(text)
    
    match textObj.language.code:
        case "ar":
            return "arabic"
        case "az":
            return "azerbaijani"
        case "eu":
            return "basque"
        case "bn":
            return "bengali"
        case "ca":
            return "catalan"
        case "zh":
            return "chinese"
        case "da":
            return "danish"
        case "nl":
            return "dutch"
        case "en":
            return "english"
        case "fi":
            return "finnish"
        case "fr":
            return "french"
        case "de":
            return "german"
        case "el":
            return "greek"
        case "he":
            return "hebrew"
        case "hi":
            return "hinglish"
        case "hu":
            return "hungarian"
        case "id":
            return "indonesian"
        case "it":
            return "italian"
        case "kk":
            return "kazakh"
        case "ne":
            return "nepali"
        case "no":
            return "norwegian"
        case "pt":
            return "portuguese"
        case "ro":
            return "romanian"
        case "ru":
            return "russian"
        case "sl":
            return "slovene"
        case "es":
            return "spanish"
        case "sv":
            return "swedish"
        case "tg":
            return "tajik"
        case "tr":
            return "turkish"
        case _:
            return "other"






def get_website_text(url_address):
    """
    Funkcija prebere besedilo strani in ga reducira na najmanjše možno obliko
    """

    with urllib.request.urlopen(url_address) as url:
        html = url.read()
        

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = ' '.join(chunk for chunk in chunks if chunk)
    sentence = text.lower()
   
    # poišče jezik vnešenega teksta iz prvih 100 besed
    reduced_sent = " ".join(sentence.split()[:100])
    lang = recognizeLang(reduced_sent)


    # creates tokens, removes stopwords and punctation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    if not lang == "other":    
        filtered_words = [w for w in tokens if not w in stopwords.words(lang)]
    else:
        filtered_words =  [w for w in tokens]

    # odstrani podvojitve
    reduced_text = list(set(filtered_words))
    #print(reduced_text)

    return " ".join(reduced_text)



# dobljena s pomočjo chatGPT
def jaccard_similarity(url1, url2):
    """
    Calculate the Jaccard similarity coefficient between the text content of two websites.
    """
    # Get the text content of each website
    text1 = get_website_text(url1)
    text2 = get_website_text(url2)
    
    # Convert the text content to sets of words
    words1 = set(text1.split())
    words2 = set(text2.split())
    
    # Calculate the Jaccard similarity coefficient
    intersection = len(words1.intersection(words2))
    union = len(words1.union(words2))
    jaccard_similarity = intersection / union
    
    return jaccard_similarity



# This is an example of code which extracts all text from a web page and compares its content
url_address1 = "http://www.google.com"
url_address2 = "http://www.google.com/search"
jaccard_similarity(url_address1, url_address2)

1.0